In [114]:
import pandas as pd
import numpy as np
import random

import seaborn as sns
import matplotlib.pyplot as plt

from solver import Instance, Solver
from argparse import Namespace
from gurobipy import Model, GRB, tupledict
import json

import ast

In [115]:
city = "berlin"
demand_baseline = "1.00"
demand_type = "uniform"

demand_file = f'{city}_db={demand_baseline}_dt={demand_type}.csv'

In [116]:
demand_df = pd.read_csv(demand_file, index_col=0)

In [117]:
# PARAMETERS
# Regions
R = sorted(list(demand_df['region_id'].unique()))

region_area_map = {}
for r in R:
    for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()):
        region_area_map[a] = r
        
# Areas 
A = {}
area_map = {a: i for i, a in enumerate(list(demand_df['area_id'].unique())) }
areas = list(area_map.keys())
n_areas = len(areas)
for r in R:
    A[r] = [ area_map[a] for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()) ]

# Employees
n_employees = 12
employees = [e for e in range(n_employees)]
E = {}
#for r in R:
#    E[r] = employees
E[0] = [0,1,2]
E[1] = [3,4,5]
E[2] = [6,7,8]
E[3] = [9,10,11]
 
# Set of all shifts available
P = {}
n_shifts = 1
for r in R:
    P[r] = list(range(n_shifts))

# WE NEED TO SET starting and end time for each shift!! and ADD THE CONSTRAINTS so no one can work before or after this time!

# Periods
n_periods = 8
Theta = [i for i in range(n_periods)]

# Days
n_days = 7
D = [d for d in range(n_days)]

# Set of demand scenarios


# Hours in shift
h = {}
n_hours = 8
for p in range(n_shifts):
    h[p] = n_hours

# Cost outsource
c_out = 1.5

# Number of deliveries to perform (n_{a theta d})
deliveries = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == {a} & day == {d}')
        deliveries[i, :, k] = ast.literal_eval(period_demands['demand'].values[0])

# Number of couriers needed (m_{a theta d})
couriers_needed = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == {a} & day == {d}')
        couriers_needed[i, :, k] = ast.literal_eval(period_demands['required_couriers'].values[0])

# Cost of employed courier (c_{a theta d})
cost_couriers = np.zeros((n_areas, n_periods, n_days))
cost_courier = 1
for i, a in enumerate(areas):
    for j, t in enumerate(Theta):
        for k, d in enumerate(D):
            cost_couriers[i, j, k] = cost_courier

# Min hours worked for employee e
min_hours_worked = 8*3
h_min = {e: min_hours_worked for e in employees}

# Max hours worked for employee e
max_hours_worked = 8*6
h_max = {e: max_hours_worked for e in employees}

# Max differing starts
max_unique_starts = 2
b_max = {e: max_unique_starts for e in employees}

# mega_value
M = 9999

# DECISION VARIABLES

In [118]:
m = Model()

# r_{e p d}
r_var = m.addVars(n_employees, n_shifts, n_days, vtype=GRB.BINARY, name='r')

# k_{e a theta d}
k_var = m.addVars(n_employees, n_areas, n_periods, n_days, vtype=GRB.BINARY, name='k')

# U_{e p}
u_var = m.addVars(n_employees, n_shifts, vtype=GRB.BINARY, name='u')

# omega_{a theta d}
omega_var = m.addVars(n_areas, n_periods, n_days, vtype=GRB.CONTINUOUS, lb=0, name='omega')

# CONSTRAINTS

In [119]:
# 1. Connecting employees moving around areas to shift assignment p
for r in R:
    for e in E[r]:
        for p in P[r]:
            for d in D:
                Ar = A[r]
                m.addConstr(
                    (sum([k_var[e, a, theta, d] for a in Ar for theta in Theta]) == 1/2 * h[p] * r_var[e,p,d] ),
                    name = f'moving_areas_{r}_{e}_{p}_{d}'          
                )

In [120]:
# 2. Employee can only be assigned to one area at a time 
for r in R:
    for e in E[r]:
        for theta in Theta:
            for d in D:
                Ar = A[r]
                m.addConstr((sum([k_var[e, a, theta, d] for a in Ar]) <= 1),
                    name = f'employee_{r}_{e}_{theta}_{d}'          
                )

# ADDED: They only can be assigned one region!
#for e in employees:
#    for theta in Theta:
#        for d in D:
#            m.addConstr((sum([k_var[e, area_map[a], theta, d] for a in areas]) <= 1),
#                name = f'only_one_area_{e}_{theta}_{d}'          
#            )

In [121]:
# 3. Employee can only work one shift a day
for r in R:
    for es in E[r]:
        for d in D:
            Pr = P[r]
            m.addConstr((sum([r_var[e, p, d] for p in Pr]) <= 1),
                name = f'only_one_shift_{r}_{e}_{theta}_{d}'          
            )

In [122]:
# 4. One rest day a week
for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([r_var[e, p, d] for p in Pr for d in D]) <= 6),
            name = f'rest_day_{r}_{e}'          
        )

In [123]:
# 5. Min - Max hours worked per week
for r in R:
    for e in E[r]:
        min_hours = h_min[e]
        max_hours = h_max[e]

        m.addConstr(( h[p] * sum([r_var[e, p, d] for p in Pr for d in D]) >= min_hours),
            name = f'min_hours_{r}_{e}'          
        )

        m.addConstr(( h[p] * sum([r_var[e, p, d] for p in Pr for d in D]) <= max_hours),
            name = f'max_hours_{r}_{e}'          
        )

In [124]:
# 6. Different shifting times constraint
for r in R:
    for e in E[r]:
        for p in P[r]:
            m.addConstr((sum([r_var[e, p, d] for d in D]) <= u_var[e, p] * M),
                name = f'start_times_{r}_{e}_1'          
            )

for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([r_var[e, p, d] for p in Pr]) <= b_max[e]),
            name = f'start_times_{r}_{e}_2'          
        )

In [125]:
# 7. Employee can't work then outsource
for r in R:
    for a in A[r]:
        for theta in Theta:
            for d in D:
                if couriers_needed[a,theta,d] > 0:
                    factor = deliveries[a,theta,d] / couriers_needed[a,theta,d] #* c_out WE COMPARE NUMBER OF PEOPLE
                else:
                    factor = 0
                m.addConstr(
                    #(couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]]) * factor <= omega_var[a, theta, d]),
                    (couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]]) <= omega_var[a, theta, d]),
                    name = f'start_times_{r}_{e}_2'   
                )

# OBJECTIVE FUNCTION

In [126]:
factor_s = 1
m.setObjective(
    sum([
        sum([cost_couriers[a,theta,d] * k_var[e,a,theta,d] for e in E[r]]) 
            + omega_var[a,theta,d]
    for r in R for a in A[r] for theta in Theta for d in D]
    )
)

In [127]:
m.ModelSense = GRB.MINIMIZE
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4204 rows, 43048 columns and 33568 nonzeros
Model fingerprint: 0x613a508b
Variable types: 3304 continuous, 39744 integer (39744 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 2863.0000000
Presolve removed 3081 rows and 39315 columns
Presolve time: 0.08s
Presolved: 1123 rows, 3733 columns, 9511 nonzeros
Found heuristic solution: objective 2840.0000000
Variable types: 0 continuous, 3733 integer (3727 binary)

Root relaxation: cutoff, 982 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cuto

# SOLUTION

In [128]:
# Total couriers in day
couriers_needed_ = []
for d in D:
    data = (
        pd.DataFrame(couriers_needed[:,:,d], index=areas, columns=Theta).reset_index()
        .rename(columns={'index': 'area'})
        #.assign(day = d)
    )

    area_day_data = (
        pd.melt(data, id_vars=['area'], value_vars=set(data.columns).difference('area'))
        .rename(columns={'variable': 'period', 'value': 'n_couriers'})
        .assign(day = d)
    )
    couriers_needed_.append(area_day_data)

couriers_needed_df = pd.concat(couriers_needed_)
couriers_needed_df

,area,period,n_couriers,day
0,10115,0,1.0,0
1,10117,0,0.0,0
2,10119,0,1.0,0
3,10178,0,1.0,0
4,10179,0,1.0,0
...,...,...,...,...
467,10965,7,1.0,6
468,10967,7,1.0,6
469,10969,7,1.0,6
470,10997,7,1.0,6


In [139]:
# Employee shifts
employees_shifts = []
for e in employees:
    for p in range(n_shifts):
        for d in D:
            value = r_var[e, p, d].X
            if value > 0.5:
                hours_worked = h[p]
                employees_shifts.append([e, p, d, hours_worked])

employees_shifts_df = pd.DataFrame(employees_shifts, columns=['employee', 'shift', 'day', 'hours_worked'])
employees_shifts_df

,employee,shift,day,hours_worked
0,0,0,0,8
1,0,0,2,8
2,0,0,6,8
3,1,0,1,8
4,1,0,4,8
5,1,0,6,8
6,2,0,3,8
7,2,0,4,8
8,2,0,6,8
9,3,0,0,8


In [141]:
# Total hours worked
employees_shifts_df.groupby('employee')['hours_worked'].sum()

employee
0     24
1     24
2     24
3     24
4     24
5     24
6     24
7     24
8     24
9     24
10    24
11    24
Name: hours_worked, dtype: int64

In [130]:
# Area Employee assignment
area_employee_assignment = []
for e in employees:
    for a in areas:
        for theta in Theta:
            for d in D:
                value = k_var[e, area_map[a],theta,d].X
                if value > 0.5:
                    area_employee_assignment.append([e, a, theta, d])

area_employee_assignment_df = pd.DataFrame(area_employee_assignment, columns=['employee', 'area', 'period', 'day'])
area_employee_assignment_df

,employee,area,period,day
0,0,10247,3,0
1,0,10247,6,6
2,0,10247,7,2
3,0,10315,0,6
4,0,10315,2,0
...,...,...,...,...
139,11,10715,1,3
140,11,10719,2,1
141,11,10719,7,1
142,11,10777,5,5


In [131]:
(
    area_employee_assignment_df
    .query('employee == 0')
    .merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'day', 'period', 'area', 'region', 'n_couriers']]
    .head(100)
)

,employee,day,period,area,region,n_couriers
6,0,0,1,10319,0,1.0
4,0,0,2,10315,0,1.0
0,0,0,3,10247,0,1.0
5,0,0,7,10317,0,1.0
8,0,2,0,10365,0,1.0
10,0,2,5,10365,0,1.0
7,0,2,6,10319,0,1.0
2,0,2,7,10247,0,1.0
3,0,6,0,10315,0,1.0
11,0,6,1,10367,0,1.0


In [132]:
(
    area_employee_assignment_df
    .query('employee == 1')
    #.merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'day', 'period', 'area', 'region']] #, 'n_couriers']]
    .head(100)
)

,employee,day,period,area,region
20,1,1,2,10367,0
21,1,1,3,10369,0
16,1,1,4,10249,0
23,1,1,7,10369,0
12,1,4,1,10247,0
22,1,4,5,10369,0
19,1,4,6,10365,0
15,1,4,7,10247,0
13,1,6,2,10247,0
14,1,6,5,10247,0


In [133]:
# areas
area_period_days_df = (
    area_employee_assignment_df
    .groupby(['area', 'period', 'day'])
    .agg({'employee': ['count', 'unique']})
    .reset_index()
)
area_period_days_df.columns = ['area', 'period', 'day', 'employee_count', 'employees_assign']
area_period_days_df

,area,period,day,employee_count,employees_assign
0,10115,0,0,1,[5]
1,10115,0,6,1,[3]
2,10115,1,0,1,[5]
3,10115,2,0,1,[5]
4,10115,4,0,1,[3]
...,...,...,...,...,...
134,10999,2,0,1,[6]
135,10999,2,6,1,[8]
136,10999,4,0,2,"[7, 8]"
137,10999,5,0,1,[7]


In [134]:
# Outsourcing
outsourcing_shifts = []
for a in areas:
    for theta in Theta:
        for d in D:
            value = omega_var[area_map[a],theta,d].X
            #print(f'{a} {theta} {d} : {value}')
            if value > 0.0:
                outsourcing_shifts.append([a, theta, d, value])

outsourcing_shifts_df = pd.DataFrame(outsourcing_shifts, columns=['area', 'period', 'day', 'n_outsource'])
outsourcing_shifts_df

,area,period,day,n_outsource
0,10115,0,1,1.0
1,10115,0,2,1.0
2,10115,0,3,1.0
3,10115,0,4,2.0
4,10115,0,5,1.0
...,...,...,...,...
2619,10999,7,2,1.0
2620,10999,7,3,1.0
2621,10999,7,4,1.0
2622,10999,7,5,1.0


In [135]:
# Join all
whole_solution_df = (
    couriers_needed_df
    # Employees
    .merge(area_period_days_df,  on=['area', 'period', 'day'], how='left')
    # Outsource
    .merge(outsourcing_shifts_df, on=['area', 'period', 'day'], how='left')
)
whole_solution_df.head(30)

,area,period,n_couriers,day,employee_count,employees_assign,n_outsource
0,10115,0,1.0,0,1.0,[5],NaN
1,10117,0,0.0,0,NaN,NaN,NaN
2,10119,0,1.0,0,NaN,NaN,1.0
3,10178,0,1.0,0,NaN,NaN,1.0
4,10179,0,1.0,0,NaN,NaN,1.0
5,10243,0,1.0,0,1.0,[8],NaN
6,10245,0,1.0,0,NaN,NaN,1.0
7,10247,0,1.0,0,NaN,NaN,1.0
8,10249,0,1.0,0,NaN,NaN,1.0
9,10315,0,1.0,0,NaN,NaN,1.0


In [136]:
whole_solution_df.query('n_couriers == 2')

,area,period,n_couriers,day,employee_count,employees_assign,n_outsource
85,10559,1,2.0,0,1.0,[9],1.0
115,10969,1,2.0,0,1.0,[6],1.0
176,10999,2,2.0,0,1.0,[6],1.0
182,10243,3,2.0,0,1.0,[6],1.0
236,10115,4,2.0,0,1.0,[3],1.0
...,...,...,...,...,...,...,...
3193,10247,6,2.0,6,1.0,[0],1.0
3196,10317,6,2.0,6,NaN,NaN,2.0
3204,10409,6,2.0,6,NaN,NaN,2.0
3207,10439,6,2.0,6,1.0,[3],1.0


In [142]:
whole_solution_df.query('n_couriers == 2 & employee_count == 2')

,area,period,n_couriers,day,employee_count,employees_assign,n_outsource
294,10999,4,2.0,0,2.0,"[7, 8]",NaN
412,10999,6,2.0,0,2.0,"[7, 8]",NaN
3063,10965,3,2.0,6,2.0,"[6, 7]",NaN
3073,10243,4,2.0,6,2.0,"[6, 8]",NaN
3084,10405,4,2.0,6,2.0,"[4, 5]",NaN


In [137]:
theta = 2
area = 10999

a = area_map[area]
d = 0
r = region_area_map[area]
# couriers_needed[a, theta, d]
print(couriers_needed[a, theta, d])

# employees
for e in E[r]:
    val = k_var[e, a, theta, d].X
    if val > 0.5:
        print(e)

print(sum([k_var[e, a, theta, d].X for e in E[r]]))

# factor
print(deliveries[a,theta,d] / couriers_needed[a,theta,d])

# outsource
print(omega_var[a, theta, d].X)

2.0
6
1.0
3.5
1.0


In [138]:
for r in R:
    for a in A[r]:
        for theta in Theta:
            for d in D:
                factor = deliveries[a,theta,d] * couriers_needed[a,theta,d] * c_out
                m.addConstr(
                    (couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]]) * factor <= omega_var[a, theta, d]),
                    name = f'start_times_{r}_{e}_2'   
                )